In [2]:
import pymatgen.core as pmg
import numpy as np

First thing we generate the bulk $\mathrm{fcc}$ structure. 
* we define the lattice with the 3 fcc vectors
* a list with a symbol for the atoms in the structure ( in this just one Al) 
* the coordinates in the same order as the symbols is a list of np.arrays of length 3

In [3]:
alat = 10.21 
lattice = np.array([[0.,1.,1.], [1.,0.,1], [1.,1.,0]])* alat *0.5
species = ['Si','Si']
coords=[np.array([0,0,0]), np.array([0.25,0.25,0.25])]

s = pmg.Structure(lattice=lattice, species=species , coords=coords, coords_are_cartesian=False) 

just to check everything is fine we make print out the info about the structure

In [4]:
s

Structure Summary
Lattice
    abc : 7.219560235914651 7.219560235914651 7.219560235914651
 angles : 60.00000000000001 60.00000000000001 60.00000000000001
 volume : 266.08306525000006
      A : 0.0 5.105 5.105
      B : 5.105 0.0 5.105
      C : 5.105 5.105 0.0
    pbc : True True True
PeriodicSite: Si (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Si (2.5525, 2.5525, 2.5525) [0.2500, 0.2500, 0.2500]

we now generate a supercell with the $z$  axis oriented along the $[1 1 0]$ direction ant the other 2 axes perpendicular to it. 
* we copy the bulk structure into s1 
* we transform `sup1` in a supercell. The  input in the second line means that the new vectors of the lattice for the supercell are $$\bf{a_1^\prime} = \bf{a_3}-\bf{a_2}  \; ; \; \bf{a_2^\prime} = \bf{a_1} \: ; \: \bf{a_3^\prime} = \bf{a_3} + \bf{a_2} -\bf{a_1}  $$

In [5]:
sup1 = s.copy()
sup1.make_supercell([[-1,-2,2],[1,-2,2],[-1,1,1]], to_unit_cell = True)
sup1

Structure Summary
Lattice
    abc : 16.14342745515958 16.14342745515958 10.21
 angles : 90.0 90.0 53.13010235415598
 volume : 2128.6645220000005
      A : 0.0 5.105 -15.315000000000001
      B : 0.0 15.315000000000001 -5.105
      C : 10.21 0.0 0.0
    pbc : True True True
PeriodicSite: Si (5.1050, 10.2100, -15.3150) [0.8750, 0.3750, 0.5000]
PeriodicSite: Si (5.1050, 5.1050, -10.2100) [0.6250, 0.1250, 0.5000]
PeriodicSite: Si (0.0000, 15.3150, -15.3150) [0.7500, 0.7500, 0.0000]
PeriodicSite: Si (0.0000, 10.2100, -10.2100) [0.5000, 0.5000, 0.0000]
PeriodicSite: Si (5.1050, 15.3150, -10.2100) [0.3750, 0.8750, 0.5000]
PeriodicSite: Si (0.0000, 5.1050, -5.1050) [0.2500, 0.2500, 0.0000]
PeriodicSite: Si (5.1050, 10.2100, -5.1050) [0.1250, 0.6250, 0.5000]
PeriodicSite: Si (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Si (7.6575, 12.7625, -12.7625) [0.6250, 0.6250, 0.7500]
PeriodicSite: Si (7.6575, 7.6575, -7.6575) [0.3750, 0.3750, 0.7500]
PeriodicSite: Si (2.5525, 17.8675, 

Now we need to transform the axes in order that $\bf{c}$ points in the $z$ direction.
The object `lattice` of pymatgen has all the info that we need. The 3 axis lengths $a,b,c$ and the angles between the axes $\alpha$ between b and c axes , $\beta$ between $\bf{a}$  and $\bf{c}$ and $\gamma$ between $\bf{a}$ and $\bf{b}$. 

We just set ${\bf c}^\prime = c \cdot \hat{\bf z}$; we then put ${\bf a}^\prime = a\cdot sin(\beta)\cdot \hat{\bf x} + a\cdot cos(\beta)\cdot\hat{\bf z}$ in the $xz$ plane forming an angle $\beta$ with ${\bf c}^\prime$; the components of $\mathbf{b}^\prime$ are found imposing that ${\bf b^\prime \cdot c^\prime} = b\,c\cdot cos(\alpha)$ and 
${\bf b^\prime \cdot a^\prime} = a\,b \cdot cos(\gamma)$

Just use this numbers to create the new vectors and print them to check we have done right

In [6]:
latt = sup1.lattice
a = latt.a ; alpha = latt.alpha*np.pi/180.0
b = latt.b ; beta = latt.beta*np.pi/180.0
c = latt.c ; gamma = latt.gamma *np.pi/180.0
print(a,b,c, alpha/np.pi*180, beta/np.pi*180, gamma/np.pi*180)
newa3 = c * np.array([0,0,1])
newa1 = a * np.array([np.sin(beta),0,np.cos(beta)])
ca = (np.cos(gamma) - np.cos(beta)*np.cos(alpha))/np.sin(beta)
cb = np.sqrt(np.sin(alpha)**2-ca**2)
newa2 = b * np.array([ca,cb,np.cos(alpha)])
newa1,newa2,newa3

16.14342745515958 16.14342745515958 10.21 90.0 90.0 53.13010235415599


(array([1.61434275e+01, 0.00000000e+00, 9.88499838e-16]),
 array([9.68605647e+00, 1.29147420e+01, 9.88499838e-16]),
 array([ 0.  ,  0.  , 10.21]))

now we replace these three axes in the structure.  then we print out structure info  to compare lengths, angles and fractional coordinates. 

In [7]:
sup1.lattice = pmg.Lattice(np.array([newa1, newa2,newa3]))
sup1

Structure Summary
Lattice
    abc : 16.14342745515958 16.14342745515958 10.21
 angles : 90.0 90.0 53.13010235415598
 volume : 2128.664522000001
      A : 16.14342745515958 0.0 9.88499838011434e-16
      B : 9.686056473095746 12.914741964127664 9.88499838011434e-16
      C : 0.0 0.0 10.21
    pbc : True True True
PeriodicSite: Si (17.7578, 4.8430, 5.1050) [0.8750, 0.3750, 0.5000]
PeriodicSite: Si (11.3004, 1.6143, 5.1050) [0.6250, 0.1250, 0.5000]
PeriodicSite: Si (19.3721, 9.6861, 0.0000) [0.7500, 0.7500, 0.0000]
PeriodicSite: Si (12.9147, 6.4574, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Si (14.5291, 11.3004, 5.1050) [0.3750, 0.8750, 0.5000]
PeriodicSite: Si (6.4574, 3.2287, 0.0000) [0.2500, 0.2500, 0.0000]
PeriodicSite: Si (8.0717, 8.0717, 5.1050) [0.1250, 0.6250, 0.5000]
PeriodicSite: Si (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Si (16.1434, 8.0717, 7.6575) [0.6250, 0.6250, 0.7500]
PeriodicSite: Si (9.6861, 4.8430, 7.6575) [0.3750, 0.3750, 0.7500]
PeriodicS

Now that the c vector is  oriented along the z cartesian axis we can build any supercell along the $[110]$ direction of any the periodicity along the xy  plane and of any thickess along the z direction 

the first two vectors define the periodicity in the plane , the third vector the thickness. For example a $2\times2$ surface with thickness 3 is generated like this:

* first we generate the supercell 

In [8]:
sup2 = sup1.copy()
sup2.make_supercell([[1,0,0],[0,1,0],[0,0,2]], to_unit_cell = False)
sup2

Structure Summary
Lattice
    abc : 16.14342745515958 16.14342745515958 20.42
 angles : 90.0 90.0 53.13010235415598
 volume : 4257.329044000002
      A : 16.14342745515958 0.0 9.88499838011434e-16
      B : 9.686056473095746 12.914741964127664 9.88499838011434e-16
      C : 0.0 0.0 20.42
    pbc : True True True
PeriodicSite: Si (17.7578, 4.8430, 5.1050) [0.8750, 0.3750, 0.2500]
PeriodicSite: Si (17.7578, 4.8430, 15.3150) [0.8750, 0.3750, 0.7500]
PeriodicSite: Si (11.3004, 1.6143, 5.1050) [0.6250, 0.1250, 0.2500]
PeriodicSite: Si (11.3004, 1.6143, 15.3150) [0.6250, 0.1250, 0.7500]
PeriodicSite: Si (19.3721, 9.6861, 0.0000) [0.7500, 0.7500, 0.0000]
PeriodicSite: Si (19.3721, 9.6861, 10.2100) [0.7500, 0.7500, 0.5000]
PeriodicSite: Si (12.9147, 6.4574, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Si (12.9147, 6.4574, 10.2100) [0.5000, 0.5000, 0.5000]
PeriodicSite: Si (14.5291, 11.3004, 5.1050) [0.3750, 0.8750, 0.2500]
PeriodicSite: Si (14.5291, 11.3004, 15.3150) [0.3750, 0.8750, 0.7500]

* then we extract the coordinates and order them along the z direction, in case we have different species we need to sort also the symbols .... 

In [9]:
coords = sup2.cart_coords
species = sup2.species
temp  = sorted(zip(species,coords), key= lambda x: x[1][2])
species = [_[0] for _ in temp ]
coords  = [_[1] for _ in temp ]
coords

[array([0., 0., 0.]),
 array([6.45737098e+00, 3.22868549e+00, 4.94249919e-16]),
 array([1.29147420e+01, 6.45737098e+00, 9.88499838e-16]),
 array([1.93721129e+01, 9.68605647e+00, 1.48274976e-15]),
 array([4.84302824, 6.45737098, 2.5525    ]),
 array([11.30039922,  9.68605647,  2.5525    ]),
 array([14.52908471,  3.22868549,  2.5525    ]),
 array([17.7577702 , 12.91474196,  2.5525    ]),
 array([17.7577702 ,  4.84302824,  5.105     ]),
 array([11.30039922,  1.61434275,  5.105     ]),
 array([14.52908471, 11.30039922,  5.105     ]),
 array([8.07171373, 8.07171373, 5.105     ]),
 array([3.22868549, 1.61434275, 7.6575    ]),
 array([16.14342746,  8.07171373,  7.6575    ]),
 array([9.68605647, 4.84302824, 7.6575    ]),
 array([22.60079844, 11.30039922,  7.6575    ]),
 array([ 6.45737098,  3.22868549, 10.21      ]),
 array([ 0.  ,  0.  , 10.21]),
 array([19.37211295,  9.68605647, 10.21      ]),
 array([12.91474196,  6.45737098, 10.21      ]),
 array([ 4.84302824,  6.45737098, 12.7625    ]),
 

* then we modify the periodicit along the a3 axes. We replace previous $\bf{c}$ vector with one that is longer than the lattice periodity in that direction this create a gap between the slabs. 

In [10]:
a1 = sup2.lattice.matrix[0]
a2 = sup2.lattice.matrix[1]
vacuum = 10 # length of the vacuum region 
a3 = sup2.lattice.matrix[2]+np.array([0.,0., vacuum])
surface = pmg.Structure(lattice = np.array([a1,a2,a3]), coords =coords, species= species, coords_are_cartesian=True)
from	collections import deque
it = (f"{_.specie.name}   {_.coords[0]:12.6f}  {_.coords[1]:12.6f}  {_.coords[2]:12.6f} \n" for _ in surface.sites) 
celldm1 = np.sqrt(surface.lattice.matrix[0].dot(surface.lattice.matrix[0]))
celldm3 = np.sqrt(surface.lattice.matrix[2].dot(surface.lattice.matrix[2]))/celldm1 
celldm4 = surface.lattice.matrix[0].dot(surface.lattice.matrix[1])/celldm1**2  
with open('posizioniSi100_8layer','w') as f:
	f.write(f"ibrav = 12  celldm(1)={celldm1:9.6f} celldm(3) =  {celldm3:9.6f} celldm(4) = {celldm4:9.6f}\n")
	s = [" ".join((f"{round(_,6):12.6}" for _ in surface.lattice.matrix[i])) for i in [0,1,2]]    
	f.write ( "\n".join(s) + "\n")
	deque((f.write(_) for _ in it )) 
 

In [11]:
surface.to(filename='surf.cif')

In [12]:
p = surface.sites[4]

In [13]:
surface.lattice.matrix

array([[1.61434275e+01, 0.00000000e+00, 9.88499838e-16],
       [9.68605647e+00, 1.29147420e+01, 9.88499838e-16],
       [0.00000000e+00, 0.00000000e+00, 3.04200000e+01]])

In [14]:
a = surface.lattice.matrix 
print (f"alat = { np.sqrt(a[0].dot(a[0]))} cosgamma={a[0].dot(a[1])/a[1].dot(a[1])} ")

alat = 16.14342745515958 cosgamma=0.6 


In [16]:
np.arccos(0.6)/np.pi**2

0.093954649073803